In [186]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
from tqdm import tqdm 

%config IPCompleter.greedy=True

In [2]:
polish_corpora_path = '../../../polish_corpora.txt'
poleval2_path = '../../../poleval_2grams.txt'
poleval3_path = '../../../poleval_3grams.txt'

## Sentences

In [3]:
sentences = []
with open('sentences.txt', encoding='utf8') as f:
    for line in f:
        line = line.strip().lower().split()
        sentences.append(line)
        
print(len(sentences))

68


## Unigrams

In [5]:
unigrams: dict = {}  # word -> number of occurrences

with open(polish_corpora_path, encoding="utf8") as f:
    for line in tqdm(f, desc='Loading data...', position=0, leave=True):
        line = line.strip().lower().split()
        for word in line:
            _word = bytes(bytearray(word, 'UTF-8'))
            if _word in unigrams:
                unigrams[_word] += 1
            else:
                unigrams[_word] = 1

Loading data...: 23011601it [05:50, 65686.98it/s]


In [6]:
print(f'Number of unigrams: {len(unigrams)}')

Number of unigrams: 3591114


In [12]:
all_unigrams = sum([x for x in unigrams.values()])
all_unigrams

451846640

## Bigrams

In [7]:
bigrams: dict = {}  # word -> List[Tuple[word_after, number of occurrences]]
    
with open(poleval2_path, encoding="utf8") as f:
    for line in tqdm(f, desc='Loading data...', position=0, leave=True):
        line = line.strip().lower().split()
        key: bytearray = bytes(bytearray(line[1], 'UTF-8'))
        value: tuple = (bytes(bytearray(line[2], 'UTF-8')), line[0])
            
        if key in bigrams:
            bigrams[key].append(value)
        else:
            bigrams[key] = [value]

Loading data...: 59134224it [02:15, 437635.80it/s]


In [8]:
print(f'Number of bigrams: {len(bigrams)}')

Number of bigrams: 3591115


## Counting probability

In [245]:
def simple_pbb_bigrams(w1: bytes, w2: bytes) -> int:
    """ Calculate number of occurrences of bigram (w1, w2) """
    assert type(w1) == bytes and type(w2) == bytes
  
    if w1 in bigrams:
        for word_after_w1, n_ocurr in bigrams[w1]:
            if word_after_w1 == w2:
                return int(n_ocurr)
    return 0


def cond_pbb_bigrams(w1: bytes, w2: bytes) -> float:
    """ Calculate P(w2 | w1) """
    assert type(w1) == bytes and type(w2) == bytes
    cnt_w1_w2: int = 1
    if w1 in bigrams:
        for word_after_w1, n_ocurr in bigrams[w1]:
            if word_after_w1 == w2:
                cnt_w1_w2 = int(n_ocurr)
                break
        
    cnt_w1: int = 1
    if w1 in unigrams:
        cnt_w1 = unigrams[w1]
        
    return cnt_w1_w2 / cnt_w1


def lambda_cond_pbb_bigrams(
    w1: bytes, w2: bytes, 
    lambda1: float, lambda2: float
) -> float:
    """ Calculate P*(w2 | w1) """
    assert type(w1) == bytes and type(w2) == bytes
    cnt_w1_w2: int = 1
    if w1 in bigrams:
        for word_after_w1, n_ocurr in bigrams[w1]:
            if word_after_w1 == w2:
                cnt_w1_w2 = int(n_ocurr)
                break
        
    cnt_w1: int = 1
    if w1 in unigrams:
        cnt_w1 = unigrams[w1]
        
    return lambda1 * pbb_unigram(w2) + lambda2 * (cnt_w1_w2 / cnt_w1)


def pbb_unigram(w1: bytes) -> float:
    """ Calculate P(w1) """
    assert type(w1) == bytes
    cnt_w1 = 1
    if w1 in unigrams:
        cnt_w1 = unigrams[w1]
        
    return cnt_w1 / all_unigrams


def pbb_sentence(sentence: list, cond_pbb_func) -> float:
    """ Calculate P(w1 ... wn ) """
    n: int = len(sentence)
    assert n > 0
    pbb = pbb_unigram(w1=sentence[0])
    for i in range(1, n):
        pbb *= cond_pbb_func(w1=sentence[i - 1], w2=sentence[i])
        
    return pbb

## Functions

In [220]:
def generate_permutations(sentence: list) -> list:
    return list(itertools.permutations(sentence))

def order_permutations(permutations: list, scoring_function, cond_pbb_func) -> list:
    scores = []
    for x in tqdm(permutations, desc='Scoring permutations...', leave=True, position=0):
        scores.append(scoring_function(x, cond_pbb_func))
    return sorted(zip(scores, permutations), reverse=True)

def score_permuatations(valid_sentence: list, scoring_function, cond_pbb_func) -> float:
    permutations: list = generate_permutations(sentence=valid_sentence)
    permutations = order_permutations(
        permutations=permutations, scoring_function=scoring_function, cond_pbb_func=cond_pbb_func
    )
    permutations = [y for x, y in permutations]
    best_decoded = list(map(lambda x: x.decode('utf8'), permutations[0]))
    print(f'Best: {best_decoded}')
    for i, sentence in enumerate(permutations):
        if tuple(sentence) == tuple(valid_sentence):
            return 1 / (i + 1)
    return 0

def final_score(cond_pbb_func) -> float:
    total_score: float = 0.0
    for i, sentence in enumerate(sentences):
        time.sleep(1)
        if len(sentence) > 7:
            print(f'{i+1} / {len(sentences)} | {sentence} ===> ??? / 1.0')
            print('Skipped because sentence is too long.')
            continue
        sentence_bytes = list(map(lambda x: bytes(bytearray(x, encoding='UTF-8')), sentence))
        score = score_permuatations(
            valid_sentence=sentence_bytes, 
            scoring_function=pbb_sentence,
            cond_pbb_func=cond_pbb_func
        )
        total_score += score
        print(f'{i+1} / {len(sentences)} | {sentence} ===> {score:.3f} / 1.0')
    return score

## $P(w_1 ... w_n) = P(w_1)*P(w_2 | w_1) * ... * P(w_n | w_{n-1})$
## $P(w2 | w1) = \frac{cnt(w1w2)}{cnt(w1)}$

In [216]:
%%time
print(f'Final score: {final_score(cond_pbb_func=cond_pbb_bigrams)}')

Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 1469.73it/s]


Best: ['wczoraj', 'stefanowi', 'judyta', 'czekoladki', 'dała']
1 / 68 | ['judyta', 'dała', 'wczoraj', 'stefanowi', 'czekoladki'] ===> 0.011 / 1.0


Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 418.62it/s]


Best: ['dwa', 'koziołki', 'rogate', 'babuleńka', 'miała']
2 / 68 | ['babuleńka', 'miała', 'dwa', 'rogate', 'koziołki'] ===> 0.010 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 1520.32it/s]


...

Best: ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu']
66 / 68 | ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu'] ===> 1.000 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 1575.04it/s]


Best: ['zanim', 'rycerze', 'księżniczkę', 'osaczyli', 'strawił', 'smoka']
67 / 68 | ['rycerze', 'osaczyli', 'smoka', 'zanim', 'strawił', 'księżniczkę'] ===> 0.003 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:06<00:00, 109.44it/s]

Best: ['już', 'chyba', 'należy', 'przestać', 'wymyślać', 'zdania']
68 / 68 | ['chyba', 'należy', 'już', 'przestać', 'wymyślać', 'zdania'] ===> 0.062 / 1.0
Final score: 0.0625
CPU times: user 39min 15s, sys: 5.12 s, total: 39min 20s
Wall time: 40min 26s


In [247]:
print(f'True accuracy: {(0.0625 * 68) / 65 :.2f}')

True accuracy: 0.07


About 7 were correct a few were really close. <br>
**22 35 40 42 53 55 66**

## $P(w_1 ... w_n) = P(w_1)*P(w_2 | w_1) * ... * P(w_n | w_{n-1})$
## $P^*(w2 | w1) = \lambda_1 * P(w2) + \lambda_2 * P(w2 | w1)$
## $P(w2 | w1) = \frac{cnt(w1w2)}{cnt(w1)}$

In [238]:
def find_lambdas() -> tuple:
    lambda1, lambda2 = 0.0, 0.0
    iters = 0
    for word, successor_words in tqdm(bigrams.items(), desc='Calculating lambdas ...'):
        for w2, n_occur in successor_words:
            pbb_w1 = pbb_unigram(w1=word)
            pbb_w1_w2 = cond_pbb_bigrams(w1=word, w2=w2)
            if pbb_w1_w2 >= pbb_w1:
                lambda2 += int(n_occur)
            else:
                lambda1 += int(n_occur)
        iters  += 1
        if iters > 100:
            break
    return lambda1, lambda2

In [239]:
lambda1, lambda2 = find_lambdas()

lambda1, lambda2

Calculating lambdas ...:   0%|          | 13/3591115 [02:41<12373:36:53, 12.40s/it]


KeyboardInterrupt: 

In [240]:
lambda1, lambda2 = 0.1, 0.9

In [242]:
%%time
f_score = final_score(
    cond_pbb_func=lambda w1, w2: lambda_cond_pbb_bigrams(
        w1=w1, w2=w2, lambda1=lambda1, lambda2=lambda2
    )
)
print(f'Final score: {f_score}')

Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 1461.21it/s]


Best: ['wczoraj', 'stefanowi', 'czekoladki', 'judyta', 'dała']
1 / 68 | ['judyta', 'dała', 'wczoraj', 'stefanowi', 'czekoladki'] ===> 0.011 / 1.0


Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 667.20it/s]


Best: ['dwa', 'koziołki', 'babuleńka', 'rogate', 'miała']
2 / 68 | ['babuleńka', 'miała', 'dwa', 'rogate', 'koziołki'] ===> 0.011 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 1486.63it/s]


...

Best: ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu']
66 / 68 | ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu'] ===> 1.000 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 2268.43it/s]


Best: ['zanim', 'rycerze', 'strawił', 'osaczyli', 'księżniczkę', 'smoka']
67 / 68 | ['rycerze', 'osaczyli', 'smoka', 'zanim', 'strawił', 'księżniczkę'] ===> 0.003 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:05<00:00, 137.27it/s]

Best: ['już', 'chyba', 'należy', 'przestać', 'wymyślać', 'zdania']
68 / 68 | ['chyba', 'należy', 'już', 'przestać', 'wymyślać', 'zdania'] ===> 0.067 / 1.0
Final score: 0.06666666666666667
CPU times: user 38min 20s, sys: 4.82 s, total: 38min 25s
Wall time: 39min 30s


7 correct <br>
**35 40 42 44 53 55 66**

## Multiply number of occurrences of each bigram

In [246]:
%%time
f_score = final_score(cond_pbb_func=simple_pbb_bigrams)
print(f'Final score: {f_score}')

Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 1473.18it/s]


Best: ['wczoraj', 'stefanowi', 'judyta', 'dała', 'czekoladki']
1 / 68 | ['judyta', 'dała', 'wczoraj', 'stefanowi', 'czekoladki'] ===> 0.016 / 1.0


Scoring permutations...: 100%|██████████| 120/120 [00:00<00:00, 660.60it/s]


Best: ['rogate', 'miała', 'koziołki', 'dwa', 'babuleńka']
2 / 68 | ['babuleńka', 'miała', 'dwa', 'rogate', 'koziołki'] ===> 0.009 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 1310.17it/s]


...

Best: ['rachunek', 'prawdopodobieństwa', 'w', 'życiu', 'się', 'przydaje']
66 / 68 | ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu'] ===> 0.002 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:00<00:00, 2927.27it/s]


Best: ['zanim', 'strawił', 'smoka', 'rycerze', 'osaczyli', 'księżniczkę']
67 / 68 | ['rycerze', 'osaczyli', 'smoka', 'zanim', 'strawił', 'księżniczkę'] ===> 0.002 / 1.0


Scoring permutations...: 100%|██████████| 720/720 [00:04<00:00, 158.14it/s]

Best: ['zdania', 'wymyślać', 'przestać', 'należy', 'już', 'chyba']
68 / 68 | ['chyba', 'należy', 'już', 'przestać', 'wymyślać', 'zdania'] ===> 0.001 / 1.0
Final score: 0.0014367816091954023
CPU times: user 38min 17s, sys: 4.34 s, total: 38min 21s
Wall time: 39min 25s


## Trigrams

In [ ]:
trigrams: dict = {}  # Tuple[w1, w2] -> List[Tuple[w3, number of occurrences]]
total_iters, skipped1, skipped2 = 0, 0, 0

with open(poleval3_path, encoding="utf8") as f:
    for line in tqdm(f, desc='Loading data...', position=0, leave=True):
        total_iters += 1
        line = line.strip().lower().split()

        if len(line) != 4:
            skipped1 += 1
            continue
            
        if int(line[0]) == 1:
            skipped2 += 1
            continue
            
        key: tuple = (bytes(bytearray(line[1], 'UTF-8')),  bytes(bytearray(line[2], 'UTF-8')))
        value: tuple = (bytes(bytearray(line[3], 'UTF-8')), line[0])
 
        if key in trigrams:
            trigrams[key].append(value)
        else:
            trigrams[key] = [value]